In [ ]:
import itertools
import json
import os
import pathlib
import shutil

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as kr

from datetime import datetime

SEED = 1234
tf.random.set_seed(SEED)

Defines the data paths

In [ ]:
PATH_BASE = pathlib.Path("./")

PATH_DATA_RAW = PATH_BASE / "data_raw"
PATH_DATA = PATH_BASE / "data"

PATH_DATA_RAW_TRAINING = PATH_DATA_RAW / "training"
PATH_DATA_RAW_OUTPUT = PATH_DATA_RAW / "train_gt.json"

PATH_DATA_TRAINING = PATH_DATA / "training"

PATH_DATA_TRAINING_CLASSES = [
	PATH_DATA_TRAINING / "no_mask",
	PATH_DATA_TRAINING / "all_mask",
	PATH_DATA_TRAINING / "some_mask"
]

PATH_EXPERIMENTS = PATH_BASE / "experiments"

In [ ]:
# prepares data folders
os.makedirs(PATH_DATA_TRAINING, exist_ok=True)

for p in PATH_DATA_TRAINING_CLASSES:
	os.makedirs(p, exist_ok=True)

# loads trainings images' labels
with open(PATH_DATA_RAW_OUTPUT) as json_file:
    training_labels_raw = json.load(json_file)

# splits the images in three list by class
training_labels_raw = [(k, v) for k, v in training_labels_raw.items()]
training_labels_raw = {c: list(v) for c, v in itertools.groupby(training_labels_raw, key=lambda i: i[1])}
training_labels_raw = {k: list(map(lambda i: i[0],v)) for k, v in training_labels_raw.items()}

# copy training images in the new fs struct
for label, images in training_labels_raw.items():
	for image in images:
		shutil.copy2(PATH_DATA_RAW_TRAINING / image, PATH_DATA_TRAINING_CLASSES[label])
		# on filesystem without cow support use symlink instead of

In [ ]:
dataset_generator_training = kr.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.3,
    width_shift_range=10,
    height_shift_range=10,
    shear_range=0.15,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode="nearest",
    rescale=1/255.,
    validation_split = 0.2
)

dataset_generator_test = kr.preprocessing.image.ImageDataGenerator(rescale=1/255.)

In [ ]:
dataset_training = dataset_generator_training.flow_from_directory(
    PATH_DATA_TRAINING,
    class_mode = "categorical",
    subset="training",
    seed=SEED,
    shuffle=True,
    batch_size=32)

dataset_validation = dataset_generator_training.flow_from_directory(
    PATH_DATA_TRAINING,
    class_mode = "categorical",
    subset="validation",
    seed=SEED,
    shuffle=True,
    batch_size=32)

In [ ]:
plt.figure(figsize=(10, 10))
images, labels = dataset_training.next()
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow((images[i] * 255.).astype("uint8"))
    plt.title(list(dataset_training.class_indices.keys())[list(dataset_training.class_indices.values()).index(np.argmax(labels[i]))])
    plt.axis("off")

## Model

In [ ]:
dataset_training.image_shape

In [ ]:
input_shape=(None,*dataset_training.image_shape)
output_shape=dataset_training.num_classes

depth = 5
start_f = 8

def filters_from_depth(d:int) -> int:
    return start_f*(2**d)

model = kr.Sequential()

for i in range(depth):

    model.add(kr.layers.Conv2D(
        filters=filters_from_depth(i),
        kernel_size=(3,3),
        strides=(1,1),
        padding="same"
    ))
    model.add(kr.layers.ReLU())
    model.add(kr.layers.MaxPool2D())

model.add(kr.layers.Flatten())
model.add(kr.layers.Dense(units=512, activation=kr.activations.relu))
#model.add(kr.layers.Dropout(0.3))
model.add(kr.layers.Dense(units=output_shape, activation=kr.activations.softmax))

model.build(input_shape=input_shape)

In [ ]:
model.compile(
    optimizer=kr.optimizers.Adam(learning_rate=1e-4),
    loss=kr.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [ ]:
model.summary()

In [ ]:
PATH_CURRENT_EXPERIMENT = PATH_EXPERIMENTS / datetime.now().strftime('%b%d_%H-%M-%S')

PATH_CHECKPOINTS = PATH_CURRENT_EXPERIMENT / "checkpoints"
PATH_TENSORBOARD_LOG = PATH_CURRENT_EXPERIMENT / "tb_log"

os.makedirs(PATH_CURRENT_EXPERIMENT, exist_ok=True)
os.makedirs(PATH_TENSORBOARD_LOG, exist_ok=True)

callbacks = [
    kr.callbacks.TensorBoard(PATH_TENSORBOARD_LOG,
                             histogram_freq=1,
                             profile_batch=0),
    kr.callbacks.ModelCheckpoint(PATH_CHECKPOINTS),
    kr.callbacks.EarlyStopping(patience=5)
]

In [ ]:
model.fit(
    dataset_training,
    validation_data=dataset_validation,
    epochs=100,
    steps_per_epoch=dataset_training.batch_size,
    callbacks=callbacks
)

In [ ]:
#avoid this method due to https://github.com/tensorflow/tensorflow/pull/44769 still open

# training_labels = list(map(lambda i: i[1], sorted(training_labels_raw.items(), key=lambda x: x[0])))
#
# training_dataset = tf.keras.preprocessing.image_dataset_from_directory(
#     training_image_dir,
#     labels = training_labels,
#     label_mode = "categorical",
#     validation_split = 0.2,
#     subset="training",
#     seed=123,
#     image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
#     batch_size=32)
